<a href="https://colab.research.google.com/github/imyjim/AltStore/blob/main/templates/aistudio_gemini_prompt_freeform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

### Install & import

In [1]:
!pip install -U -q google-generativeai

In [2]:
# Install the client library and import necessary modules.
import google.generativeai as genai

import base64
import copy
import hashlib
import io
import json
import mimetypes
import pathlib
import pprint
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

## Set the API key

Add your API_KEY to the secrets manager in the left panel "🔑".

In [4]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")

SecretNotFoundError: Secret GOOGLE_API_KEY does not exist.

In [ ]:
# Configure the client library by providing your API key.
genai.configure(api_key=GOOGLE_API_KEY)

## Parse the arguments

In [ ]:
model = 'gemini-1.5-pro-002' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6IkdlbmVyYXRlIGEgRG9ja2VyIHNjcmlwdCB0byBjcmVhdGUgYSBzaW1wbGUgbGludXggbWFjaGluZSB0aGF0IGhhcyBweXRob24gMy4xMCAocHJlYnVpbHQpIGluc3RhbGxlZCB3aXRoIGZvbGxvd2luZyBsaWJyYXJpZXM6IHBhbmRhcywgdGVuc29yZmxvdywgbnVtcHkifV19LHsicm9sZSI6Im1vZGVsIiwicGFydHMiOlt7InRleHQiOiJgYGBkb2NrZXJmaWxlXFxuIyBVc2UgYSBiYXNlIGltYWdlIHdpdGggUHl0aG9uIDMuMTBcXG5GUk9NIHB5dGhvbjozLjEwXFxuXFxuIyBVcGRhdGUgdGhlIHBhY2thZ2UgbGlzdCBhbmQgaW5zdGFsbCBuZWNlc3NhcnkgZGVwZW5kZW5jaWVzXFxuUlVOIGFwdC1nZXQgdXBkYXRlICYmIGFwdC1nZXQgaW5zdGFsbCAteSBcXFxcXFxuICAgIGJ1aWxkLWVzc2VudGlhbCBcXFxcXFxuICAgIGxpYm9wZW5ibGFzLWJhc2UgXFxcXFxcbiAgICBsaWJsYXBhY2stZGV2IFxcXFxcXG4gICAgZysrXFxuXFxuIyBJbnN0YWxsIFB5dGhvbiBsaWJyYXJpZXNcXG5SVU4gcGlwIGluc3RhbGwgcGFuZGFzIHRlbnNvcmZsb3cgbnVtcHlcXG5cXG4jIEV4cG9zZSB0aGUgZGVmYXVsdCBQeXRob24gcG9ydFxcbkVYUE9TRSA4MDAwXFxuXFxuIyBTZXQgYSBkZWZhdWx0IGNvbW1hbmQgdG8gc3RhcnQgYW4gaW50ZXJhY3RpdmUgUHl0aG9uIHNoZWxsXFxuQ01EIFtcXFwicHl0aG9uXFxcIl1cXG5gYGBcXG5cXG4qKkV4cGxhbmF0aW9uOioqXFxuXFxuMS4gKipGUk9NIHB5dGhvbjozLjEwOioqIFRoaXMgbGluZSBkZWZpbmVzIHRoZSBiYXNlIGltYWdlIGFzIHRoZSBvZmZpY2lhbCBQeXRob24gMy4xMCBEb2NrZXIgaW1hZ2UuIEl0IHByb3ZpZGVzIGEgbWluaW1hbCBMaW51eCBlbnZpcm9ubWVudCB3aXRoIFB5dGhvbiAzLjEwIHByZWluc3RhbGxlZC5cXG4yLiAqKlJVTiBhcHQtZ2V0IHVwZGF0ZSAmJiBhcHQtZ2V0IGluc3RhbGwgLXkgLi4uOioqIFRoaXMgc3RlcCB1cGRhdGVzIHRoZSBwYWNrYWdlIGxpc3QgYW5kIGluc3RhbGxzIGVzc2VudGlhbCBzeXN0ZW0gZGVwZW5kZW5jaWVzLiBJdCBpbnN0YWxsczpcXG4gICAtICoqYnVpbGQtZXNzZW50aWFsOioqIFByb3ZpZGVzIGVzc2VudGlhbCB0b29scyBmb3IgYnVpbGRpbmcgc29mdHdhcmUgZnJvbSBzb3VyY2UgY29kZS5cXG4gICAtICoqbGlib3BlbmJsYXMtYmFzZSBhbmQgbGlibGFwYWNrLWRldjoqKiBOZWVkZWQgZm9yIFRlbnNvckZsb3cgYW5kIE51bVB5IHRvIHdvcmsgcHJvcGVybHkuXFxuICAgLSAqKmcrKzoqKiBBIEMrKyBjb21waWxlciB1c2VkIGJ5IHNvbWUgUHl0aG9uIGxpYnJhcmllcy5cXG4zLiAqKlJVTiBwaXAgaW5zdGFsbCBwYW5kYXMgdGVuc29yZmxvdyBudW1weToqKiAgSW5zdGFsbHMgdGhlIHNwZWNpZmllZCBQeXRob24gbGlicmFyaWVzIChwYW5kYXMsIFRlbnNvckZsb3csIGFuZCBOdW1QeSkgdXNpbmcgcGlwLlxcbjQuICoqRVhQT1NFIDgwMDA6KiogRXhwb3NlcyB0aGUgZGVmYXVsdCBwb3J0ICg4MDAwKSBvZnRlbiB1c2VkIGJ5IHdlYiBhcHBsaWNhdGlvbnMgcnVubmluZyB3aXRoIFB5dGhvbi4gWW91IGNhbiBjaGFuZ2UgdGhpcyBpZiBuZWVkZWQuXFxuNS4gKipDTUQgW1xcXCJweXRob25cXFwiXToqKiAgU2V0cyBhIGRlZmF1bHQgY29tbWFuZCB0byBydW4gdGhlIGludGVyYWN0aXZlIFB5dGhvbiBzaGVsbCB3aGVuIHlvdSBydW4gdGhlIGNvbnRhaW5lci5cXG5cXG4qKkhvdyB0byB1c2UgdGhlIERvY2tlcmZpbGU6KipcXG5cXG4xLiAqKlNhdmUgdGhlIGNvZGUgYWJvdmUgYXMgYSBmaWxlIG5hbWVkIGBEb2NrZXJmaWxlYCBpbiB5b3VyIHByb2plY3QgZGlyZWN0b3J5LioqXFxuMi4gKipPcGVuIGEgdGVybWluYWwgYW5kIG5hdmlnYXRlIHRvIHRoZSBkaXJlY3RvcnkgY29udGFpbmluZyBgRG9ja2VyZmlsZWAuKipcXG4zLiAqKkJ1aWxkIHRoZSBpbWFnZToqKiBgZG9ja2VyIGJ1aWxkIC10IHB5dGhvbi1lbnYgLmAgIChSZXBsYWNlIGBweXRob24tZW52YCB3aXRoIHlvdXIgZGVzaXJlZCBpbWFnZSBuYW1lKVxcbjQuICoqUnVuIHRoZSBjb250YWluZXI6KiogYGRvY2tlciBydW4gLWl0IHB5dGhvbi1lbnYgYmFzaGAgKFRoaXMgc3RhcnRzIGEgYmFzaCBzaGVsbCBpbnNpZGUgdGhlIGNvbnRhaW5lci4pXFxuNS4gKipZb3UgY2FuIG5vdyBpbXBvcnQgYW5kIHVzZSB0aGUgaW5zdGFsbGVkIFB5dGhvbiBsaWJyYXJpZXMgd2l0aGluIHRoZSBjb250YWluZXIncyBlbnZpcm9ubWVudC4qKlxcblxcbioqVG8gY3VzdG9taXplIGZ1cnRoZXI6KipcXG5cXG4qICoqQWRkIG90aGVyIGxpYnJhcmllczoqKiBJbmNsdWRlIHRoZW0gaW4gdGhlIGBwaXAgaW5zdGFsbGAgY29tbWFuZC5cXG4qICoqU2V0IHNwZWNpZmljIGVudmlyb25tZW50IHZhcmlhYmxlczoqKiBVc2UgdGhlIGBFTlZgIGRpcmVjdGl2ZSBpbiB0aGUgRG9ja2VyZmlsZS5cXG4qICoqQ2hhbmdlIHRoZSBkZWZhdWx0IGNvbW1hbmQ6KiogTW9kaWZ5IHRoZSBgQ01EYCBpbnN0cnVjdGlvbi5cXG4qICoqTW91bnQgdm9sdW1lczoqKiBBbGxvdyBhY2Nlc3MgdG8gZGF0YSBvciBmaWxlcyBvbiB5b3VyIGhvc3QgbWFjaGluZSB3aXRoaW4gdGhlIGNvbnRhaW5lciB1c2luZyB0aGUgYC12YCBmbGFnIHdoZW4gcnVubmluZyB0aGUgY29udGFpbmVyLlxcbiogKipVc2UgZGlmZmVyZW50IGJhc2UgaW1hZ2VzOioqIEZvciBzcGVjaWFsaXplZCBuZWVkcywgZXhwbG9yZSBhbHRlcm5hdGl2ZSBiYXNlIGltYWdlcyBiYXNlZCBvbiBQeXRob24sIFVidW50dSwgRGViaWFuLCBvciBvdGhlcnMuXFxuIn1dfSx7InJvbGUiOiJ1c2VyIiwicGFydHMiOlt7InRleHQiOiJSVU4gYXB0LWdldCB1cGRhdGUgJiYgYXB0LWdldCBpbnN0YWxsIC15IFxcXFxcXG4gICAgYnVpbGQtZXNzZW50aWFsIFxcXFxcXG4gICAgbGlib3BlbmJsYXMtYmFzZSBcXFxcXFxuICAgIGxpYmxhcGFjay1kZXYgXFxcXFxcbiAgICBnKysifV19LHsicm9sZSI6Im1vZGVsIiwicGFydHMiOlt7InRleHQiOiJUaGlzIHBhcnQgb2YgdGhlIERvY2tlcmZpbGUgaGFuZGxlcyBpbnN0YWxsaW5nIHN5c3RlbSBkZXBlbmRlbmNpZXMgZm9yIFB5dGhvbiBsaWJyYXJpZXMgbGlrZSBUZW5zb3JGbG93IGFuZCBOdW1QeSwgd2hpY2ggb2Z0ZW4gcmVxdWlyZSBsb3ctbGV2ZWwgbGlicmFyaWVzIGZvciBvcHRpbWFsIHBlcmZvcm1hbmNlLiBcXG5cXG5MZXQncyBicmVhayBkb3duIGVhY2ggZWxlbWVudDpcXG5cXG4qKmBSVU4gYXB0LWdldCB1cGRhdGUgJiYgYXB0LWdldCBpbnN0YWxsIC15IC4uLmA6KipcXG5cXG4tICoqYGFwdC1nZXQgdXBkYXRlYDoqKiBSZWZyZXNoZXMgdGhlIHBhY2thZ2UgbGlzdCBmb3IgdGhlIERlYmlhbi1iYXNlZCBMaW51eCBlbnZpcm9ubWVudCAoY29tbW9uIGZvciBEb2NrZXIgaW1hZ2VzKS4gIFRoaXMgZW5zdXJlcyB5b3UgaGF2ZSBhY2Nlc3MgdG8gdGhlIG1vc3QgcmVjZW50IHNvZnR3YXJlIHZlcnNpb25zLlxcbi0gKipgYXB0LWdldCBpbnN0YWxsIC15IC4uLmA6KiogSW5zdGFsbHMgdGhlIHNwZWNpZmllZCBwYWNrYWdlcywgYXV0b21hdGljYWxseSByZXNwb25kaW5nIFxcXCJ5ZXNcXFwiIHRvIGFueSBwcm9tcHRzLiBUaGlzIHNpbXBsaWZpZXMgdGhlIGluc3RhbGxhdGlvbiBwcm9jZXNzIGFuZCBwcmV2ZW50cyBuZWVkaW5nIHRvIG1hbnVhbGx5IGFwcHJvdmUgcHJvbXB0cy5cXG4tICoqYC4uLmA6KiogVGhlIGAuLi5gIHJlZmVycyB0byB0aGUgYWN0dWFsIHBhY2thZ2UgbmFtZXMgdG8gaW5zdGFsbDpcXG4gICAgLSAqKmBidWlsZC1lc3NlbnRpYWxgOioqICBBIG1ldGEtcGFja2FnZSBwcm92aWRpbmcgZXNzZW50aWFsIHRvb2xzIGZvciBidWlsZGluZyBzb2Z0d2FyZSBmcm9tIHNvdXJjZSBjb2RlLiBJdCBpbmNsdWRlcyBjb21waWxlcnMgbGlrZSBHQ0MgKEMvQysrKSBhbmQgYmFzaWMgYnVpbGQgdXRpbGl0aWVzIChsaWtlIGBtYWtlYCkuIFRlbnNvckZsb3cgYW5kIE51bVB5IG1heSByZWx5IG9uIHRoaXMuXFxuICAgIC0gKipgbGlib3BlbmJsYXMtYmFzZWA6KiogQSBoaWdoLXBlcmZvcm1hbmNlIEJMQVMgKEJhc2ljIExpbmVhciBBbGdlYnJhIFN1YnByb2dyYW1zKSBpbXBsZW1lbnRhdGlvbi4gSXQgcHJvdmlkZXMgaGlnaGx5IG9wdGltaXplZCBtYXRoZW1hdGljYWwgZnVuY3Rpb25zIHRoYXQgc3BlZWQgdXAgY29tcHV0YXRpb25zLlxcbiAgICAtICoqYGxpYmxhcGFjay1kZXZgOioqIENvbnRhaW5zIHJvdXRpbmVzIGZvciBzb2x2aW5nIGxpbmVhciBlcXVhdGlvbnMgYW5kIHBlcmZvcm1pbmcgb3RoZXIgbWF0cml4IGNvbXB1dGF0aW9ucy4gVXNlZCBpbiBjb21iaW5hdGlvbiB3aXRoIEJMQVMgYnkgbGlicmFyaWVzIGxpa2UgVGVuc29yRmxvdyBhbmQgTnVtUHkuXFxuICAgIC0gKipgZysrYDoqKiBUaGUgR05VIEMrKyBjb21waWxlci4gTmVlZGVkIGZvciBjb21waWxpbmcgY2VydGFpbiBwYXJ0cyBvZiBQeXRob24gbGlicmFyaWVzIGlmIHRoZXkgaW52b2x2ZSBDL0MrKyBleHRlbnNpb25zLiBcXG5cXG4qKkltcG9ydGFuY2Ugb2YgdGhlc2UgcGFja2FnZXM6KipcXG5cXG4tICoqUGVyZm9ybWFuY2U6KiogQkxBUyBhbmQgTEFQQUNLIGFyZSBjcml0aWNhbCBmb3Igc2NpZW50aWZpYyBjb21wdXRpbmcuIE9wdGltaXplZCBpbXBsZW1lbnRhdGlvbnMgc2lnbmlmaWNhbnRseSBhY2NlbGVyYXRlIG1hdHJpeCBvcGVyYXRpb25zIGFuZCBtYXRoZW1hdGljYWwgZnVuY3Rpb25zIGNvbW1vbmx5IHVzZWQgaW4gUHl0aG9uIGRhdGEgc2NpZW5jZSBsaWJyYXJpZXMuXFxuLSAqKkJ1aWxkaW5nIGRlcGVuZGVuY2llczoqKiBNYW55IFB5dGhvbiBsaWJyYXJpZXMgKGxpa2UgVGVuc29yRmxvdykgYXJlIGJ1aWx0IHdpdGggZXh0ZW5zaW9ucyB3cml0dGVuIGluIEMvQysrLiBUaGUgYnVpbGQgdG9vbHMgcHJvdmlkZWQgYnkgYGJ1aWxkLWVzc2VudGlhbGAgKGluY2x1ZGluZyBgZysrYCkgYXJlIG5lZWRlZCB0byBjb21waWxlIHRob3NlIHBhcnRzIGNvcnJlY3RseS4gXFxuXFxuKipOb3RlOioqXFxuXFxuLSBUaGlzIGluc3RhbGxhdGlvbiBkZXBlbmRzIG9uIHRoZSBjaG9zZW4gYmFzZSBEb2NrZXIgaW1hZ2UgKGxpa2UgYHB5dGhvbjozLjEwYCkuIFlvdSdsbCBuZWVkIHRvIGNob29zZSBhIERlYmlhbi1iYXNlZCBiYXNlIGltYWdlIGZvciB0aGlzIHNwZWNpZmljIGBhcHQtZ2V0YCBpbnN0YWxsYXRpb24gdG8gd29yayBjb3JyZWN0bHkuIFxcbi0gU29tZSBiYXNlIGltYWdlcyBtaWdodCBhbHJlYWR5IGhhdmUgdGhlc2UgZGVwZW5kZW5jaWVzIHByZS1pbnN0YWxsZWQuIENvbnNpZGVyIGNoZWNraW5nIHRoZSBiYXNlIGltYWdlIGRvY3VtZW50YXRpb24uIFxcbi0gSXQncyBpbXBvcnRhbnQgdG8gdXBkYXRlIHRoZSBwYWNrYWdlIGxpc3QgdXNpbmcgYGFwdC1nZXQgdXBkYXRlYCAqKmJlZm9yZSoqIGluc3RhbGxpbmcgcGFja2FnZXMgdG8gZW5zdXJlIHlvdSBnZXQgdGhlIGxhdGVzdCBhdmFpbGFibGUgdmVyc2lvbnMuXFxuIn1dfV0=' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo0MCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

In [ ]:
gais_contents

In [ ]:
generation_config

In [ ]:
safety_settings

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [ ]:
# @title `show_file` function
drive = None
def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")

In [ ]:
for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")

## Convert & upload files

For each file, Google AI Studio either sent:
- a Google Drive IDs
- a URL, or
- the raw bytes (`inline_data`).

The API itself onlty understands two ways of sending files:

- `inline_data` - where the bytes are placed inline in the request.
- `file_data` - where the file is uploaded to the Files API, and you pass a reference to that file.

This section goes through the `contents` from Google AI Studio, and uploads the file data, to the Files API, so Gemini can access it:

In [ ]:
# @title `upload_file` function

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)

drive = None
def upload_file_data(file_data):
    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        hash = hashlib.sha256(data).hexdigest()
        path = tempfiles / hash
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files"in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

In [ ]:
contents = copy.deepcopy(gais_contents)

for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data)

Here is the coverted `Content`s:

In [ ]:
contents

## Call `generate_content`

In [ ]:
# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=False,
)

In [ ]:
if generation_config.get("candidate_count", 1) == 1:
    display(Markdown(response.text))

In [ ]:
response.candidates

## Or Create a chat

In [ ]:
gemini = genai.GenerativeModel(
    model_name=model,
    generation_config=generation_config,
    safety_settings=safety_settings,
)

A `ChatSession`, should always end with the model's turn.

So before creating the `chat` check whos turn was last.

If the user was last, attach all but the last content as the `history` and send the last content with `send_message` to get the model's response.

If the model was last, put the whole contents list in the history.

In [ ]:
model_turn = contents[-1].get("role", None) == "user"

if model_turn:
    chat = gemini.start_chat(history=contents[:-1])

    response = chat.send_message(contents[-1])

    if generation_config.get("candidate_count", 1) == 1:
        display(Markdown(response.text))
else:
    chat = gemini.start_chat(history=contents)

In [ ]:
if model_turn:
    response.candidates

After that use `send_message` to continue the conversation.

In [ ]:
# response = chat.send_message("Interesting, tell me more.")
# display(Markdown(response.text))